In [1]:
import os
import re
import torch
from torch.utils.data import Dataset, DataLoader, random_split

from data_loader_v1 import create_dataloader_v1
from clean_gutenberg_text import clean_gutenberg_text

## Cleaning single novels

In [2]:
def concatinate_text_data(directory="original_texts/"):
    all_text = ""
    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            text = clean_gutenberg_text(os.path.join(directory, filename))
            all_text += text + "\n<|endoftext|>\n"
            
    # Replace multiple consecutive <|endoftext|> with a single one
    all_text = re.sub(r"(<\|endoftext\|>\n*)+", "<|endoftext|>\n", all_text, flags=re.DOTALL)
                
    return all_text

In [3]:
text_data = concatinate_text_data()

In [4]:
# Train/validation ratio
train_ratio = 0.90
split_idx = int(train_ratio * len(text_data))
train_data = text_data[:split_idx]
val_data = text_data[split_idx:]

with open('train_text_data.txt', 'w') as f:
    f.write(train_data)
with open('val_text_data.txt', 'w') as f:
    f.write(val_data)

## Cleaning Gutenburg's collected works of Jane Austen

In [2]:
from clean_gutenberg_collected_work import clean_gutenberg_collected_work

In [3]:
directory = "original_texts/"
cleaned_text = clean_gutenberg_collected_work(os.path.join(directory, "complete_jane_austen.txt"))

In [4]:
# Train/validation ratio
train_ratio = 0.90
split_idx = int(train_ratio * len(cleaned_text))
train_data = cleaned_text[:split_idx]
val_data = cleaned_text[split_idx:]

with open('train_text_data.txt', 'w') as f:
    f.write(train_data)
with open('val_text_data.txt', 'w') as f:
    f.write(val_data)